# `CatBoost` vs `Light GBM` vs `XGBoost`

In [1]:
!pip install -q hvplot
!pip install scikit-learn
!pip install xgboost
!pip install lightgbm
!pip install catboost


[notice] A new release of pip is available: 24.0 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
import pandas as pd

data = pd.read_csv('comparison/data.csv')
data.loc[data.grad_rate > 100, 'grad_rate'] = 100

X = data.drop('private', axis=1)
y = data.private.map({"Yes": 1, "No": 0})
print(data.columns)
print()

print(X)
print(y)

Index(['private', 'apps', 'accept', 'enroll', 'top10perc', 'top25perc',
       'f_undergrad', 'p_undergrad', 'outstate', 'room_board', 'books',
       'personal', 'phd', 'terminal', 's_f_ratio', 'perc_alumni', 'expend',
       'grad_rate'],
      dtype='object')

      apps  accept  enroll  top10perc  top25perc  f_undergrad  p_undergrad  \
0     1660    1232     721         23         52         2885          537   
1     2186    1924     512         16         29         2683         1227   
2     1428    1097     336         22         50         1036           99   
3      417     349     137         60         89          510           63   
4      193     146      55         16         44          249          869   
..     ...     ...     ...        ...        ...          ...          ...   
772   2197    1515     543          4         26         3089         2029   
773   1959    1805     695         24         47         2849         1107   
774   2097    1915     695        

In [5]:
data.private.value_counts()

private
Yes    565
No     212
Name: count, dtype: int64

# XGBoost vs LightGBM vs CatBoost

In [53]:
import numpy as np
import pandas as pd
from time import time
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

def test_model(X, y, model_name):
    
    if model_name == 'XGBoost':
        model = XGBClassifier()
    elif model_name == 'LightGBM':
        model = LGBMClassifier(verbose=-1)  # вербоус -1 щоб не спамило
    elif model_name == 'CatBoost':
        model = CatBoostClassifier(verbose=0)  # вербоус 0 щоб не спамило
    else:
        raise ValueError("Unsupported model name.")
    
    # Збереження результатів
    accuracy = {}
    speed_train = {}
    speed_predict = {}
    std_scores = {}
    
    # Розділення даних на тренувальні та тестові
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Використання крос-валідації для оцінки моделі
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=42)
    
    # Тренування та вимірювання швидкості навчання
    start = time()
    score = cross_val_score(model, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)
    train_time = np.round(time() - start, 3)
    
    # Вимірювання швидкості передбачення
    model.fit(X_train, y_train)
    start = time()
    y_pred = model.predict(X_test)
    predict_time = np.round(time() - start, 3)
    
    # Збереження результатів
    accuracy[model_name] = (np.mean(score) * 100).round(3)
    speed_train[model_name] = train_time
    speed_predict[model_name] = predict_time
    std_scores[model_name] = np.std(score).round(3)
    
    # Вивід результатів
    print(f"{model_name} Model:")
    print(f"Mean F1 Score: {accuracy[model_name]}")
    print(f"STD of F1 Scores: {std_scores[model_name]}")
    print(f"Training Time: {speed_train[model_name]}s")
    print(f"Prediction Time: {speed_predict[model_name]}s\n")
    
    return accuracy, speed_train, speed_predict, std_scores

In [66]:
# Зберігання результатів
all_accuracy = []
all_speed_train = []
all_speed_predict = []
all_std_scores = []

# Тест кожної моделі
for model_name in ['XGBoost', 'LightGBM', 'CatBoost']:
    accuracy, speed_train, speed_predict, std_scores = test_model(X, y, model_name)
    all_accuracy.append({'Algorithm': model_name, 'Accuracy': accuracy[model_name]})
    all_speed_train.append({'Algorithm': model_name, 'Training Time': speed_train[model_name]})
    all_speed_predict.append({'Algorithm': model_name, 'Prediction Time': speed_predict[model_name]})
    all_std_scores.append({'Algorithm': model_name, 'STD F1 Score': std_scores[model_name]})

XGBoost Model:
Mean F1 Score: 96.168
STD of F1 Scores: 0.012
Training Time: 0.308s
Prediction Time: 0.005s

LightGBM Model:
Mean F1 Score: 96.355
STD of F1 Scores: 0.012
Training Time: 1.041s
Prediction Time: 0.002s

CatBoost Model:
Mean F1 Score: 96.519
STD of F1 Scores: 0.01
Training Time: 25.371s
Prediction Time: 0.002s



In [67]:
# Переводимо час у мілісекунди
for entry in all_speed_predict:
    entry['Prediction Time'] *= 1000  # переводимо в мс

print(all_speed_predict)


[{'Algorithm': 'XGBoost', 'Prediction Time': 5.0}, {'Algorithm': 'LightGBM', 'Prediction Time': 3.0}, {'Algorithm': 'CatBoost', 'Prediction Time': 2.0}]


In [2]:
print(all_speed_train)
all_speed_predict = [{'Algorithm': 'XGBoost', 'Prediction Time': 5.0}, {'Algorithm': 'LightGBM', 'Prediction Time': 3.0}, {'Algorithm': 'CatBoost', 'Prediction Time': 1.0}]

NameError: name 'all_speed_train' is not defined

In [1]:
# Фрейми для графіків
accuracy_df = pd.DataFrame(all_accuracy)
speed_train_df = pd.DataFrame(all_speed_train)
speed_predict_df = pd.DataFrame(all_speed_predict)
std_scores_df = pd.DataFrame(all_std_scores)

# Побудова графіків
accuracy_plot = accuracy_df.hvplot.barh(x='Algorithm', y='Mean F1', title="Mean F1 Score By Model")
speed_train_plot = speed_train_df.hvplot.barh(x='Algorithm', y='Training Time', title="Model Training Time Comparison (s)")
speed_predict_plot = speed_predict_df.hvplot.barh(x='Algorithm', y='Prediction Time', title="Model Prediction Time Comparison (ms)")
std_scores_plot = std_scores_df.hvplot.barh(x='Algorithm', y='STD F1 Score', title="Model STD F1 Score Comparison")

# Вивід графіків
accuracy_plot + speed_train_plot + speed_predict_plot + std_scores_plot

NameError: name 'pd' is not defined

In [10]:
import pandas as pd
import hvplot.pandas

accuracy_df = []
accuracy_df.append({'Algorithm': "XBoost", 'Mean F1 Score': 96.168})
accuracy_df.append({'Algorithm': "LightGBM", 'Mean F1 Score': 96.168})
accuracy_df.append({'Algorithm': "CatBoost", 'Mean F1 Score': 96.168})

accuracy_df = pd.DataFrame(accuracy_df)

accuracy_plot = accuracy_df.hvplot.barh(x='Algorithm', y='Mean F1 Score', title="Mean F1 Score By Model")
accuracy_plot + accuracy_plot

:Layout
   .Bars.I  :Bars   [Algorithm]   (Mean F1 Score)
   .Bars.II :Bars   [Algorithm]   (Mean F1 Score)